In [ ]:
!pip install langchain openai duckduckgo-search tweepy smtplib pywhatkit

In [ ]:
import os
import smtplib
import pywhatkit as kit
import tweepy
from duckduckgo_search import DDGS
from langchain.chat_models import ChatOpenAI

In [ ]:
# --- SETUP YOUR KEYS ---

# OpenAI Key
os.environ["OPENAI_API_KEY"] = "your-openai-key-here"

In [ ]:
# Twitter API keys
twitter_consumer_key = "your-twitter-consumer-key"
twitter_consumer_secret = "your-twitter-consumer-secret"
twitter_access_token = "your-twitter-access-token"
twitter_access_secret = "your-twitter-access-secret"

In [ ]:
# Email Sender credentials
email_sender = "your_email@gmail.com"
email_password = "your_email_password"

In [ ]:
# WhatsApp default sending time (set after 1-2 mins)
import datetime
now = datetime.datetime.now()

In [ ]:
# --- Initialize models ---
llm = ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo")

In [ ]:
# --- Define functions ---

def search_ai_paper(topic):
    with DDGS() as ddgs:
        results = ddgs.text(topic + " AI research paper", max_results=1)
        for result in results:
            return result["href"], result["body"]
    return None, None

def summarize_text(text):
    prompt = f"Summarize the following AI research paper in simple terms:\n\n{text}"
    return llm.predict(prompt)

def send_tweet(message):
    auth = tweepy.OAuth1UserHandler(
        twitter_consumer_key, twitter_consumer_secret,
        twitter_access_token, twitter_access_secret
    )
    api = tweepy.API(auth)
    api.update_status(message)
    print("🐦 Tweeted successfully!")

def send_email(receiver_email, subject, message):
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(email_sender, email_password)
            email_message = f"Subject: {subject}\n\n{message}"
            server.sendmail(email_sender, receiver_email, email_message)
            print("📧 Email sent successfully!")
    except Exception as e:
        print("Error sending email:", e)

def send_whatsapp(phone_number, message):
    try:
        kit.sendwhatmsg(phone_number, message, now.hour, now.minute + 2)
        print("🟢 WhatsApp message scheduled!")
    except Exception as e:
        print("Error sending WhatsApp:", e)



In [ ]:
# --- Main Agent Logic ---

def agentic_ai_system():
    topic = input("🔍 Enter AI research topic to search: ")
    email = input("📧 Enter your email (or press Enter to skip): ").strip()
    phone = input("📱 Enter your WhatsApp phone number (or press Enter to skip): ").strip()

    print("\nSearching for research paper...")
    url, paper_content = search_ai_paper(topic)

    if not url:
        print("❌ Could not find any paper.")
        return

    print(f"\nFound Paper: {url}")
    print("\nSummarizing...")
    summary = summarize_text(paper_content)
    print("\n🧠 Summary:\n", summary)

    # Step 4: Conditional actions
    if len(summary) <= 160:
        send_tweet(summary)
    else:
        print("✖️ Summary too long to tweet.")

    if email:
        send_email(email, f"Summary of {topic}", summary)

    if phone:
        send_whatsapp(phone, summary)

In [ ]:
# --- Run the Agent ---
if __name__ == "__main__":
    agentic_ai_system()